In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import pathlib


import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split





## Important pathes

In [4]:
BASE_PATH = "/kaggle/input/plant-pathology-2020-fgvc7"
IMAGES_PATH = f"{BASE_PATH}/images/"
TRAIN_PATH = f"{BASE_PATH}/trian.csv"
TEST_PATH = f"{BASE_PATH}/test.csv"
SUBMISSION_PATH = f"{BASE_PATH}/sample_submission.csv"

datasets_folder = os.path.join("/kaggle/input/plant-pathology-2020-fgvc7")

## Loading Train dataset

In [5]:
# load train dataset
train_df = pd.read_csv(os.path.join(datasets_folder, "train.csv"))
train_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


## Checking images Path

In [6]:
image_dataset = pathlib.Path(os.path.join(datasets_folder, "images"))
print(image_dataset)
list_images = list(image_dataset.glob('*.*'))
print(list_images[0])

# not a string 
# type(list_images[0])

/kaggle/input/plant-pathology-2020-fgvc7/images
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1743.jpg


## Splitting Columns of the dataset

In [7]:
images_id_csv = train_df['image_id']
healthy_csv = train_df['healthy']
multiple_diseases_csv = train_df['multiple_diseases']
rust_csv = train_df['rust']
scab_csv = train_df['scab']

## Reading images and labels

In [8]:
image_size = 50
images = []
labels = []

for index, image_name in tqdm(enumerate(images_id_csv), desc="creating images and labels"):
#     print(index, image_name)
    img = cv2.imread(os.path.join(datasets_folder, 'images', f'{image_name}.jpg'), cv2.IMREAD_COLOR)
    img = cv2.resize(img, (image_size, image_size))
    img = np.array(img)
    img = img / 255.0
    

    labels.append([healthy_csv[index], multiple_diseases_csv[index], rust_csv[index], scab_csv[index]])
    images.append(img)
    



creating images and labels: 1821it [01:26, 21.12it/s]


## Saving datasets images and labels into files

In [9]:
import pickle

pickle_out = open("x.pickle", "wb")
pickle.dump(images, pickle_out)
pickle_out.close()

pickle_out_labels = open("y.pickle", "wb")
pickle.dump(labels, pickle_out_labels)
pickle_out_labels.close()

## Loading them back again

In [10]:
# image_file = open("x.pickle", "rb")

images = pickle.load(open("x.pickle", "rb"))
labels = pickle.load(open("y.pickle", "rb"))

## Split Training and Test 

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(images, labels, test_size=0.2)



ytrain = np.array(ytrain).argmax(axis=1)
ytest = np.array(ytest).argmax(axis=1)
print(ytrain.shape, ytest.shape)

(1456,) (365,)


## Creating and training the Model

In [12]:
# import epochs as epochs
import tensorflow as tf
from tensorflow import keras


xtrain = np.array(xtrain).reshape(-1, image_size, image_size,3)

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(image_size, image_size,3)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(4, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(xtrain, ytrain, epochs=100)
test_loss, test_acc = model.evaluate(xtrain, ytrain)


2022-12-19 17:40:22.263870: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-12-19 17:40:22.509559: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
46/46 [==============================] - 1s 6ms/step - loss: 1.5011 - accuracy: 0.3434
Epoch 2/100
46/46 [==============================] - 0s 6ms/step - loss: 1.2885 - accuracy: 0.3777
Epoch 3/100
46/46 [==============================] - 0s 5ms/step - loss: 1.2047 - accuracy: 0.4087
Epoch 4/100
46/46 [==============================] - 0s 5ms/step - loss: 1.1911 - accuracy: 0.4087
Epoch 5/100
46/46 [==============================] - 0s 5ms/step - loss: 1.1919 - accuracy: 0.4162
Epoch 6/100
46/46 [==============================] - 0s 5ms/step - loss: 1.1790 - accuracy: 0.4341
Epoch 7/100
46/46 [==============================] - 0s 5ms/step - loss: 1.1391 - accuracy: 0.4746
Epoch 8/100
46/46 [==============================] - 0s 5ms/step - loss: 1.1551 - accuracy: 0.4389
Epoch 9/100
46/46 [==============================] - 0s 6ms/step - loss: 1.1350 - accuracy: 0.4670
Epoch 10/100
46/46 [==============================] - 0s 5ms/step - loss: 1.1323 - accuracy: 0.4732
Epoch 11/

## Final Results

In [13]:
print(test_loss, test_acc)

0.03261161595582962 0.9958791136741638
